# Data Preprocessing and Cleaning

Data cleaning is an essential step in the process of preparing data for further processing and serving into a large language model. Without data cleaning, the language model would be limited in its ability to interpret and process the data accurately. Data cleaning helps to ensure that the data is in a consistent format, free of any inconsistencies or errors that could lead to incorrect results. It also helps to reduce the amount of time needed to process the data, as it eliminates the need to manually check for any inconsistencies or errors. Additionally, data cleaning can help to improve the accuracy of the language model by removing any irrelevant or incorrect data. By taking the time to properly clean the data, the language model can more accurately interpret and process the data, leading to better results.

> 📍 Fill out the missing pieces in the source source to get everything working (indicated by `#FIXME`).

## Count tokens

> **Important note**
>
> *You do not need to manually tokenize strings before feeding texts into the model. This will be done automatically once you put instructions into the `prompt` parameter. However, you can use the `tiktoken` library to check how a string is tokenized and count the numbers of tokens to calculate the cost of an API call. Learn more [here](https://platform.openai.com/docs/introduction/tokens).*

Tokenizing text strings is an important step in natural language processing (NLP) as it helps models like GPT-3 understand the structure of a text string. Tokenizing breaks a text string into smaller pieces called tokens, which can then be analyzed and used by the model. By understanding the structure of a text string, models can better understand the meaning of the text. Additionally, tokenizing helps to determine the cost of an Azure OpenAI Service API call, as usage is priced by token. Furthermore, different models use different encodings, so it is important to tokenize text strings in the appropriate format.

`tiktoken` supports three encodings used by Azure OpenAI Service models:

| Encoding name | Azure OpenAI Service models |
| ------------- | -------------- |
| gpt2 (or r50k_base) | Most GPT-3 models |
| p50k_base | Code models, text-davinci-002, text-davinci-003 |
| cl100k_base | text-embedding-ada-002 |

Tokens in English typically range from one character to one word (e.g. "t" or "great"), though some languages may have tokens that are shorter or longer than one character or word. Spaces are usually placed at the beginning of words (e.g. " is" instead of "is " or "+"is"). You can use the Tokenizer to quickly check how a string is tokenized.

To show it briefly, we will use `tiktoken` to tokenize a text string and see how the output looks like.

In [0]:
pip install tiktoken

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 13.7 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import tiktoken

encoding = tiktoken.get_encoding("p50k_base")
encoding.encode("Hello world, this is fun!")

[15496, 995, 11, 428, 318, 1257, 0]

Write a script that shows the string tokens from an input phrase.

In [0]:
#FIXME
import tiktoken

# Inicializar la codificación (asegúrate de usar el nombre correcto del modelo)
encoding = tiktoken.get_encoding("p50k_base")

# Solicitar una frase de entrada
input_phrase = input("Introduce una frase para tokenizar: ")

# Codificar la frase en tokens
tokens = encoding.encode(input_phrase)

# Decodificar los tokens a cadenas para mostrarlos
token_strings = [encoding.decode([token]) for token in tokens]

# Mostrar los tokens
print("\nTokens:")
for i, (token, token_str) in enumerate(zip(tokens, token_strings)):
    print(f"{i + 1}. ID del Token: {token}, Cadena del Token: '{token_str}'")


Introduce una frase para tokenizar:  FIXME


Tokens:
1. ID del Token: 47084, Cadena del Token: 'FIX'
2. ID del Token: 11682, Cadena del Token: 'ME'


Let's write a function to count the number of tokens in a text string.

In [0]:
import tiktoken

def get_num_tokens_from_string(string: str, encoding_name: str='p50k_base') -> int:
    """
    Count the number of tokens in a given text string using the specified encoding.

    Args:
        string (str): The input text string.
        encoding_name (str): The name of the encoding model. Default is 'p50k_base'.

    Returns:
        int: The number of tokens in the input string.
    """
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(string)
    return len(tokens)

# Contar el número de tokens en una frase específica
phrase = "Hello World, this is fun!"
token_count = get_num_tokens_from_string(phrase)
print(f"Número total de tokens en la frase '{phrase}': {token_count}")

Número total de tokens en la frase 'Hello World, this is fun!': 7


## Clean data

Next we'll perform some light data cleaning by removing redundant whitespace and cleaning up the punctuation to prepare the data for tokenization. Again, tokenization is not required for the model to work, but it is a good practice to do so to ensure that the data is in a consistent format and that the model is able to process the data correctly. Also, it makes sure that the request is not too long for the model as the maximum number of tokens for davinci is 2048, e.g., equivalent to around 2-3 pages of text.

> **Best Practices**
> - **Replace newlines with a single space**: Unless you're embedding code, we suggest replacing newlines (\n) in your input with a single space, as we have observed inferior results when newlines are present.

In [0]:
import re

def normalize_text(string, sep_token = " \n "):
    """Normalize text by removing unnecessary characters and altering the format of words."""
    # make text lowercase
    string = re.sub(r'\s+',  ' ', string).strip()
    string = re.sub(r". ,","",string)
    # remove all instances of multiple spaces
    string = string.replace("..",".")
    string = string.replace(". .",".")
    string = string.replace("\n", "")
    string = string.strip()
    return string

Generate some data to test the cleaning function.

In [0]:
import re

def normalize_text(string, sep_token = " \n "):
    """Normalize text by removing unnecessary characters and altering the format of words."""
    # make text lowercase
    string = re.sub(r'\s+', ' ', string).strip()
    string = re.sub(r". ,", "", string)
    # remove all instances of multiple spaces
    string = string.replace("..", ".")
    string = string.replace(". .", ".")
    string = string.replace("\n", "")
    string = string.strip()
    return string

# Generamos algunos datos de ejemplo con varios formatos
texts = [
    "Este es un ejemplo de texto...  \n que contiene múltiples  espacios. \n\n   y puntuación innecesaria.",
    "Otro ejemplo de texto,  con varios   espacios  y signos de puntuación.. \n Esto es para probar la limpieza.",
    "   Ejemplo con más  ..    espacios y   caracteres   especiales.. y   saltos de linea  \n\n que deben ser  eliminados. ",
    "Texto con varias repeticiones de puntos.. y   comas,. que deben  ser limpiados. \n También saltos de línea.",
    "Algunos  caracteres    innecesarios ... \n  que necesitan   ser eliminados.. \n y corregidos."
]

# Aplicamos la función normalize_text a cada texto
normalized_texts = [normalize_text(text) for text in texts]

# Mostramos los resultados
for i, text in enumerate(normalized_texts):
    print(f"Texto {i+1}: {text}")


Texto 1: Este es un ejemplo de texto.. que contiene múltiples espacios. y puntuación innecesaria.
Texto 2: Otro ejemplo de texto, con varios espacios y signos de puntuación. Esto es para probar la limpieza.
Texto 3: Ejemplo con más . espacios y caracteres especiales. y saltos de linea que deben ser eliminados.
Texto 4: Texto con varias repeticiones de puntos. y comas,. que deben ser limpiados. También saltos de línea.
Texto 5: Algunos caracteres innecesarios .. que necesitan ser eliminados. y corregidos.


## Ejercicio 1: Validar longitud de texto antes de enviar a OpenAI

Escribe una función llamada validate_text_length que valide si un texto supera un límite de tokens especificado. Si el texto supera el límite, la función debe dividirlo en segmentos de tamaño adecuado para enviarlos al modelo de OpenAI. Usa la codificación cl100k_base.



In [0]:
import tiktoken

def validate_text_length(text, max_tokens=2048, encoding_name='cl100k_base'):
    # Codificar el texto en tokens usando la codificación especificada
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)
    
    # Si el texto no excede el límite, retornamos el texto como un solo segmento
    if len(tokens) <= max_tokens:
        return [text]
    
    # Si excede el límite, lo dividimos en segmentos adecuados
    segments = []
    for i in range(0, len(tokens), max_tokens):
        segment_tokens = tokens[i:i+max_tokens]
        segment_text = encoding.decode(segment_tokens)
        segments.append(segment_text)
    
    return segments

# Prueba de la función con un texto largo
text = "Lorem ipsum " * 500  # Texto largo
segments = validate_text_length(text)  # Llamamos a la función

# Verificamos la cantidad de segmentos y la longitud de tokens en el primer segmento
print(len(segments))  # Número de segmentos
print(len(tiktoken.get_encoding('cl100k_base').encode(segments[0])))  # Número de tokens del primer segmento

1
1001


## Ejercicio 2: Filtrar palabras prohibidas en un texto

Escribe una función llamada filter_prohibited_words que remueva palabras prohibidas de un texto antes de enviarlo a OpenAI. 

Como ejemplo queremos Eliminar palabras como: ['password', 'confidential', 'secret']. 



In [0]:
import re

def filter_prohibited_words(text, prohibited_words):
    # Reemplazar cada palabra prohibida por [REDACTED]
    for word in prohibited_words:
        # Usamos re.sub para reemplazar la palabra prohibida, con un límite de palabra completa (\b)
        text = re.sub(r'\b' + re.escape(word) + r'\b', '[REDACTED]', text, flags=re.IGNORECASE)
    return text

# Prueba de la función
text = "This document contains confidential and secret information."
prohibited_words = ['password', 'confidential', 'secret']

cleaned_text = filter_prohibited_words(text, prohibited_words)
print(cleaned_text)  # Resultado esperado: "This document contains [REDACTED] and [REDACTED] information."

This document contains [REDACTED] and [REDACTED] information.


## Ejercicio 3: Identificar idioma del texto

Crea una función detect_language que use langdetect para identificar el idioma de un texto y devuelva el código del idioma (e.g., en, es). Verifica el idioma antes de enviarlo al modelo OpenAI. Debes instalar la libreria langdetect


In [0]:
pip install langdetect

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=aebf4f99aa248959a8ee392bbf63f30c271393e4db3d2e41ac39b43b5a707c97
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from langdetect import detect

def detect_language(text):
    """Detecta el idioma de un texto."""
    # Usamos la función detect de langdetect para identificar el idioma
    return detect(text)

# Prueba de la función
text = "este é um texto de teste"
language = detect_language(text)
print(language)  # Resultado esperado: 'pt' (para portugues)


pt


## Ejercicio 5: Generar prompts efectivos

Escribe una función generate_effective_prompt que genere un prompt optimizado para OpenAI, dado un tema y un objetivo. Usa plantillas como:
“Explain [tema] in simple terms for a [audiencia].”



In [0]:
def generate_effective_prompt(topic, audience="general audience"):
    """Genera un prompt optimizado para OpenAI."""
    # Usamos una plantilla para generar el prompt
    prompt = f"Explain {topic} in simple terms for a {audience}."
    return prompt

# Prueba de la función
topic = "machine learning"
prompt = generate_effective_prompt(topic)  # Usamos el valor por defecto de audiencia
print(prompt)  # Resultado esperado: "Explain machine learning in simple terms for a general audience."

Explain machine learning in simple terms for a general audience.


## Ejercicio 6: Medir costo estimado del uso de OpenAI

Crea una función estimate_cost que calcule el costo estimado de una solicitud a OpenAI basado en el número de tokens. Supón un costo de $0.02 por 1000 tokens.



In [0]:
import tiktoken

def estimate_cost(text, encoding_name='cl100k_base', cost_per_1k_tokens=0.02):
    """Calcula el costo estimado de un texto basado en el número de tokens."""
    # Obtener la codificación de tokens
    encoding = tiktoken.get_encoding(encoding_name)
    
    # Codificar el texto en tokens
    tokens = encoding.encode(text)
    
    # Calcular el número de tokens
    num_tokens = len(tokens)
    
    # Calcular el costo estimado
    cost = (num_tokens / 1000) * cost_per_1k_tokens
    
    return cost

# Prueba de la función
text = "This is a sample text." * 100  # Texto largo
cost = estimate_cost(text)
print(f"Estimated cost: ${cost:.4f}")  # Resultado esperado: Costo estimado basado en la cantidad de tokens

Estimated cost: $0.0100


## Ejercicio 7: Limpiar texto JSON para OpenAI

Escribe una función clean_json_text que tome un JSON como entrada y limpie todas las claves y valores de caracteres no ASCII.


In [0]:
import json
import re

def clean_value(value):
    """Limpia un valor eliminando caracteres no ASCII."""
    if isinstance(value, str):
        # Usamos una expresión regular para eliminar caracteres no ASCII
        return re.sub(r'[^\x00-\x7F]+', '', value)
    return value

def clean_json_text(json_obj):
    """Limpia texto JSON eliminando caracteres no ASCII en claves y valores."""
    return {clean_value(k): clean_value(v) for k, v in json_obj.items()}

# Prueba de la función
data = {"key1": "Hello 😊", "key2": "Café and thé"}
cleaned_data = clean_json_text(data)
print(cleaned_data)  # Resultado esperado: {"key1": "Hello ", "key2": "Caf and th"}


{'key1': 'Hello ', 'key2': 'Caf and th'}
